# Using `AmpliGraph` to generate GoT Knowledge Graph Embeddings

<sub>Content of this notebook was prepared by Basel Shbita (shbita@usc.edu) as part of the class <u>DSCI 558: Building Knowledge Graphs</u> during Fall 2020 at University of Southern California (USC).</sub>

**Notes**: 
- You are supposed to write your code or modify our code in any cell starting with `# ** STUDENT CODE`.
- Much content of this notebook was borrowed from AmpliGraph tutorials

`AmpliGraph` is a suite of neural machine learning models for relational learning, a branch of machine learning that deals with supervised learning on knowledge graphs. It can be used to <u>generate stand-alone knowledge graph embeddings</u>, discover new knowledge from an existing knowledge graph and complete large knowledge graphs with missing statements.

**In this task, you will gain some hands-on experience working with Knowledge Graph Embeddings. Specifically, you will use the *TransE*, *DistMult* and *ComplEx* models to learn the embeddings of a (small) KG. You will be required to split the dataset to train and test sets, train the model, evaluate it and then generate a visualization for each model type!**

## Prepare the environment

Lets install the packages we will use

In [ ]:
!pip install -r requirements.txt

sanity check:

In [7]:
import numpy as np
import pandas as pd
import ampligraph

ampligraph.__version__

'1.3.2'

## Importing the dataset

Each relation (i.e. a triple) is in the form:`<subject, predicate, object>`

Run the following cell to load the dataset in memory with using the `load_from_csv()` utility function:

In [9]:
from ampligraph.datasets import load_from_csv

X = load_from_csv('.', 'similar_players.csv', sep=',', header=0)

Let's inspect the top triples:

In [10]:
pd.DataFrame(X, columns=['s', 'p', 'o']).head()

,s,p,o
0,/players/a/abdelal01.html,has_similar_playing_pattern,/players/b/blounco01.html
1,/players/a/abdelal01.html,has_similar_playing_pattern,/players/b/bryanma01.html
2,/players/a/abdelal01.html,has_similar_playing_pattern,/players/f/fundela01.html
3,/players/a/abdelal01.html,has_similar_playing_pattern,/players/h/hortoed01.html
4,/players/a/abdelal01.html,has_similar_playing_pattern,/players/h/hugheri02.html


Let's list the subject and object entities found in the dataset:

In [11]:
entities = np.unique(np.concatenate([X[:, 0], X[:, 2]]))
entities

array(['/players/a/abdelal01.html', '/players/a/abdulma01.html',
       '/players/a/abdulma02.html', ..., '/players/z/zizican01.html',
       '/players/z/zopfbi01.html', '/players/z/zubaciv01.html'],
      dtype=object)

.. and all of the relationships that link them.

In [12]:
relations = np.unique(X[:, 1])
relations

array(['has_similar_playing_pattern'], dtype=object)

## Defining train and test datasets

As is typical in machine learning, we need to split our dataset into training and test sets.

What differs from the standard method of randomly sampling N points to make up our test set, is that our data points are two entities linked by some relationship, and we need to take care to <u>ensure that all entities are represented in train and test sets by at least one triple</u>.

To accomplish this, `AmpliGraph` provides the `train_test_split_no_unseen` function.

In [13]:
from ampligraph.evaluation import train_test_split_no_unseen 


# we create a 10% test set split
X_train, X_test = train_test_split_no_unseen(X, test_size=int(X.shape[0]/10))

Our data is now split into train/test sets:

In [14]:
print('Train set size: ', X_train.shape)
print('Test set size: ', X_test.shape)

Train set size:  (17784, 3)
Test set size:  (1976, 3)


# Task 2.1

## Task 2.1.x.1 Training the model

`AmpliGraph` has implemented several Knoweldge Graph Embedding models (*TransE, ComplEx, DistMult, etc...*):

In [15]:
from ampligraph.latent_features import TransE, DistMult, ComplEx

Lets go through the parameters to understand what's going on:
- **k**: the dimensionality of the embedding space
- **eta ($\eta$)**: the number of negative, or false triples that must be generated at training runtime for each positive, or true triple
- **batches_count**: the number of batches in which the training set is split during the training loop. If you are having into low memory issues than settings this to a higher number may help.
- **epochs**: the number of epochs to train the model for.
- **optimizer**: the Adam optimizer, with a learning rate of 1e-3 set via the *optimizer_params* kwarg.
- **loss**: pairwise loss, with a margin of 0.5 set via the *loss_params* kwarg.
- **regularizer**: $L_p$ regularization with $p=2$, i.e. l2 regularization. $\lambda$ = 1e-5, set via the *regularizer_params* kwarg.

Now we can instantiate the model:

In [16]:
# ** STUDENT CODE 
# TODO: try different model types: TransE [2.1.1], DistMult [2.1.2], ComplEx [2.1.3]
# EmbeddingMethod = TransE
# EmbeddingMethod = DistMult
EmbeddingMethod = ComplEx

In [17]:
model = EmbeddingMethod(batches_count=100, 
                seed=0, 
                epochs=200, 
                k=150, 
                eta=5,
                optimizer='adam', 
                optimizer_params={'lr':1e-3},
                loss='multiclass_nll', 
                regularizer='LP', 
                regularizer_params={'p':3, 'lambda':1e-5}, 
                verbose=True)

### Filtering negatives

`AmpliGraph` aims to follow `scikit-learn`'s ease-of-use design philosophy and simplify everything down to `fit`, `evaluate`, and `predict` functions.

However, there are some knowledge graph specific steps we must take to ensure our model can be trained and evaluated correctly. The first of these is defining the filter that will be used to ensure that no *negative* statements generated by the corruption procedure are actually positives. This is simply done by concatenating our train and test sets. Now when negative triples are generated by the corruption strategy, we can check that they aren't actually true statements.

In [18]:
positives_filter = X

### Fitting the model

Once you run the next cell the model will train:

In [19]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

model.fit(X_train, early_stopping = False)

Average Loss:   0.011449: 100%|██████████| 200/200 [05:48<00:00,  1.74s/epoch]


## 2.1.x.2 Evaluating the model

Now it's time to evaluate our model on the test set to see how well it's performing.

For this we'll use the `evaluate_performance` function:

In [20]:
from ampligraph.evaluation import evaluate_performance

And let's look at the arguments to this function:

- `X`: the data to evaluate on. We're going to use our test set to evaluate.
- `model`: the model we previously trained.
- `filter_triples`: will filter out the false negatives generated by the corruption strategy.
- `use_default_protocol`: specifies whether to use the default corruption protocol. If True, then subj and obj are corrupted separately during evaluation.
- `verbose`: will give some nice log statements. Let's leave it on for now.

Let's run some evaluations:

In [21]:
ranks = evaluate_performance(X_test, 
                             model=model, 
                             filter_triples=positives_filter,
                             use_default_protocol=True,
                             verbose=True)

WARNING - DeprecationWarning: use_default_protocol will be removed in future. Please use corrupt_side argument instead.


100%|██████████| 1976/1976 [00:35<00:00, 55.00it/s]


The `ranks` returned by the `evaluate_performance` function indicate the rank at which the test set triple was found when performing link prediction using the model.

<u>For example</u>, if we run the triple `<House Stark of Winterfell, IN_REGION, The North>` and the model returns a rank of `7`, it tells us that while it's not the highest likelihood true statement (which would be given a rank 1), it's pretty likely.

For the evaluation metrics, we are going to use the `mrr_score` (mean reciprocal rank) and `hits_at_n_score` functions:
- `mrr_score`: The function computes the mean of the reciprocal of elements of a vector of rankings ranks.
- `hits_at_n_score`: The function computes how many elements of a vector of rankings ranks make it to the top n positions.

In [22]:
from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score

mrr = mrr_score(ranks)
print("MRR: %.2f" % (mrr))

hits_10 = hits_at_n_score(ranks, n=10)
print("Hits@10: %.2f" % (hits_10))
hits_3 = hits_at_n_score(ranks, n=3)
print("Hits@3: %.2f" % (hits_3))
hits_1 = hits_at_n_score(ranks, n=1)
print("Hits@1: %.2f" % (hits_1))

MRR: 0.22
Hits@10: 0.48
Hits@3: 0.24
Hits@1: 0.10


`Hits@N` indicates how many times in average a true triple was ranked in the top-N. The choice of which N makes more sense depends on the application. The Mean Reciprocal Rank (`MRR`) is another popular metrics to assess the predictive power of a model.

**^ Please note that a screenshot of these scores are required for task of 2.1.x.2 ^**

## Predicting New Links

Link prediction allows us to infer missing links in a graph. This has many real-world use cases, such as predicting connections between people in a social network, interactions between proteins in a biological network, and music recommendation based on prior user taste.

In our case, we are going to see which of the following candidate statements are more likely to be true:

In [56]:
X_unseen = np.array([
    ['/players/t/thompkl01.html', 'has_similar_playing_pattern', '/players/c/curryst01.html'],
    ['/players/b/bryanko01.html', 'has_similar_playing_pattern', '/players/w/wadedw01.html'],
    ['/players/t/thompkl01.html', 'has_similar_playing_pattern', '/players/b/bryanko01.html'],
    ['/players/a/anthoca01.html', 'has_similar_playing_pattern', '/players/j/jamesle01.html']
])

In [57]:
unseen_filter = np.array(list({tuple(i) for i in np.vstack((positives_filter, X_unseen))}))

In [58]:
ranks_unseen = evaluate_performance(
    X_unseen, 
    model=model, 
    filter_triples=unseen_filter,
    corrupt_side = 's+o',
    use_default_protocol=False,
    verbose=True
)

100%|██████████| 4/4 [00:00<00:00, 25.61it/s]


In [59]:
scores = model.predict(X_unseen)

We transform the scores (real numbers) into probabilities (bound between 0 and 1) using the `expit` transform (note that the probabilities are not calibrated).

In [60]:
from scipy.special import expit
probs = expit(scores)

In [31]:
probs

array([0.35542285], dtype=float32)

In [61]:
pd.DataFrame(list(zip([' '.join(x) for x in X_unseen], 
                      ranks_unseen, 
                      np.squeeze(scores),
                      np.squeeze(probs))), 
             columns=['statement', 'rank', 'score', 'prob']).sort_values("score", ascending=False)

,statement,rank,score,prob
1,/players/b/bryanko01.html has_similar_playing_...,1,10.095822,0.999959
3,/players/a/anthoca01.html has_similar_playing_...,195,1.910819,0.871111
0,/players/t/thompkl01.html has_similar_playing_...,4150,-0.595286,0.355423
2,/players/t/thompkl01.html has_similar_playing_...,3783,-0.616620,0.350551


## Task 2.1.x.3: Visualizing Embeddings with Tensorboard projector

we can now visualize the high-dimensional embeddings in the browser. Lets import the `create_tensorboard_visualization` function, which simplifies the creation of the files necessary for Tensorboard to display the embeddings.

In [22]:
from ampligraph.utils import create_tensorboard_visualizations

And now we'll run the function with our model, specifying the output path:

In [23]:
create_tensorboard_visualizations(model, 'dsci558_embeddings')

If all went well, we should now have a number of files a directory called `dsci558_embeddings`.
To visualize the embeddings in Tensorboard, go to (`cd`) `dsci558_embeddings` and run the following command: `tensorboard --logdir=./visualizations`

**^ Please note that a screenshot of embedding visualization is required for task 2.1.x.3 ^**